## Solve 1D diffusion-advection equation in transient state for 1 crack (+internal Neumann BC)

## Websites to visit:
- https://fenicsproject.discourse.group/t/integrating-over-an-interior-surface/247
- https://fenicsproject.org/qa/12118/variable-vector-field/
- https://fenicsproject.discourse.group/t/getting-cell-dofs-for-vector-valued-function-space-in-dolfinx/5167/2

In [1]:
import numpy as np
from mpi4py import MPI
import pyvista
import ufl

from ufl import Measure, FacetNormal

import dolfinx
from dolfinx import fem, plot
from dolfinx.io import XDMFFile
from dolfinx.fem import FunctionSpace, VectorFunctionSpace, Constant, Function

from petsc4py import PETSc
from petsc4py.PETSc import ScalarType

## Read the mesh

In [2]:
# Read the mesh
with XDMFFile(MPI.COMM_WORLD, "mesh_1crack_extended/1crack2D_extended_mesh.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="Grid")
    cell_tags = xdmf.read_meshtags(mesh, name="Grid")
mesh.topology.create_connectivity(mesh.topology.dim-1, mesh.topology.dim)

with XDMFFile(MPI.COMM_WORLD, "mesh_1crack_extended/1crack2D_extended_facet_mesh.xdmf", "r") as xdmf:
    facet_tags = xdmf.read_meshtags(mesh, name="Grid")

In [3]:
topology, cell_types, geometry = plot.create_vtk_mesh(mesh, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

pyvista.set_jupyter_backend("pythreejs")

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

## Define temporal parameters

In [4]:
t = 0 # Start time
T = 100.0 # Final time 1000
num_steps = 50     
dt = T / num_steps # time step size

TL = 5 # initial temperature and temperature on the right hand side

## Finite element function space

In [5]:
V = FunctionSpace(mesh, ("CG", 1))  # Lagrange element and and linear elements (degree 1)

## Set initial conditions

In [6]:
T_n = Function(V)
T_n.name = "p_n"
T_n.x.array[:] = np.full(len(T_n.x.array), TL)

## Boundary conditions

In [7]:
# DIRICHLET: T=TL on all Dirichlet BC (side 1)
boundary_dofs = fem.locate_dofs_topological(V, mesh.topology.dim-1, facet_tags.indices[facet_tags.values == 1])
bc = fem.dirichletbc(ScalarType(TL), boundary_dofs, V)

# INTERNAL NEUMANN: dp/dn=-4 on side 4
ds = Measure("dS", domain=mesh, subdomain_data=facet_tags)   # here dS and not ds !
n = FacetNormal(mesh)
g = Constant(mesh, ScalarType((50, 0)))

## Time-dependent output

In [8]:
T_h = T_n.copy()
T_h.name = "T_h"

## Trial and test functions

In [9]:
T, r = ufl.TrialFunction(V), ufl.TestFunction(V)

## Source term

In [10]:
# here f=0 as there is no source term
f = fem.Constant(mesh, ScalarType(0))

## Variational problem

In [11]:
# physical properties
rho = 1
c = 1
rho_w = 1
c_w = 1
cond = 1

In [12]:
# define constant flux in the fracture and 0 flux on the rock
Q = VectorFunctionSpace(mesh, ("DG", 0))
q = Function(Q)
num_cells = mesh.topology.index_map(mesh.topology.dim).size_local   # number of cells (here it is equal to the number of dofmap, see explanation below)
block_size = Q.dofmap.index_map_bs  # number of dof per dofmap
for i in range(num_cells):
    # CAREFUL: go see the link: https://fenicsproject.discourse.group/t/getting-cell-dofs-for-vector-valued-function-space-in-dolfinx/5167/2
    # Here each cell has only 1 dofmap as we have constant elements ("DG", 0) so Q.dofmap.cell_dofs(i) == num_dofs
    # But if we have linear elements then "num_dofs" will be the number of the cell (size 1) and Q.dofmap.cell_dofs(i) will return the 3 nodes of the cell (size 3). 
    # Then, as here we have a VectorFunctionSpace (not a FunctionSpace), we have to take into account that the block size will be 2 and not size 1 as for a normal FunctionSpace.
    # So in the end the real number of dof for the VectorFunctionSpace will be:
    # nb_cells (given by mesh.topology.index_map(mesh.topology.dim).size_local) * nb_cell_dofs (given by Q.dofmap.cell_dofs(i)) * block_size (given by Q.dofmap.index_map_bs)
    # If you want to check your understanding decomment the 2 following lines:
    # cell_blocks = Q.dofmap.cell_dofs(i)
    # print(i, cell_blocks)
    if cell_tags.values[i] == 5:
        q.x.array[[i*block_size, i*block_size+1]] = [0, 0]
    elif cell_tags.values[i] == 6:
        q.x.array[[i*block_size, i*block_size+1]] = [-60, 0]  # [-10, 0]

In [13]:
# sanity check
print(sum(q.x.array==-60))
print(sum(cell_tags.values == 6))
print("Same number so OK")

406
406
Same number so OK


In [14]:
# see the link mentioned on the top of the notebook to understand the last two terms of F
F = T * r * ufl.dx + dt * cond/(rho*c) * ufl.dot(ufl.grad(T), ufl.grad(r)) * ufl.dx + dt * (rho_w*c_w)/(rho*c) * ufl.dot(q, ufl.grad(T)) * r * ufl.dx \
    - (T_n * r * ufl.dx + dt * f * r * ufl.dx - dt * cond/(rho*c) * ufl.dot(g('+'), n('+')) * r('+') * ds(4))

In [15]:
a = ufl.lhs(F)
L = ufl.rhs(F)

## Preparation of linear algebra structures for time dependent problems

In [16]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)

In [17]:
A = fem.petsc.assemble_matrix(bilinear_form, bcs=[bc])
A.assemble()
b = fem.petsc.create_vector(linear_form)

## Linear solver

In [18]:
solver = PETSc.KSP().create(mesh.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

In [19]:
import pyvista
pyvista.set_jupyter_backend("ipygany")

grid = pyvista.UnstructuredGrid(*plot.create_vtk_mesh(V))

def plot_function(t, uh):
    """
    Create a figure of the concentration uh warped visualized in 3D at timet step t.
    """
    p = pyvista.Plotter()
    # Update point values on pyvista grid
    grid.point_data[f"u({t})"] = uh.x.array.real
    # Warp mesh by point values
    warped = grid.warp_by_scalar(f"u({t})", factor=1.5)

    # Add mesh to plotter and visualize in notebook or save as figure
    actor = p.add_mesh(warped)
    if not pyvista.OFF_SCREEN:
        p.show()
    else:
        pyvista.start_xvfb()
        figure_as_array = p.screenshot(f"diffusion_{t:.2f}.png")
        # Clear plotter for next plot
        p.remove_actor(actor)
plot_function(0, T_h)

AppLayout(children=(VBox(children=(HTML(value='<h3>u(0)</h3>'), Dropdown(description='Colormap:', options={'Br…

In [20]:
for i in range(num_steps):
    t += dt

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    fem.petsc.assemble_vector(b, linear_form)
    
    # Apply Dirichlet boundary condition to the vector
    fem.petsc.apply_lifting(b, [bilinear_form], [[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    fem.petsc.set_bc(b, [bc])

    # Solve linear problem
    solver.solve(b, T_h.vector)
    T_h.x.scatter_forward()

    # Update solution at previous time step (u_n)
    T_n.x.array[:] = T_h.x.array

    # Plot every 15th time step
    if i % 15 == 0:
        plot_function(t, T_h)

AppLayout(children=(VBox(children=(HTML(value='<h3>u(2.0)</h3>'), Dropdown(description='Colormap:', options={'…

AppLayout(children=(VBox(children=(HTML(value='<h3>u(32.0)</h3>'), Dropdown(description='Colormap:', options={…

AppLayout(children=(VBox(children=(HTML(value='<h3>u(62.0)</h3>'), Dropdown(description='Colormap:', options={…

AppLayout(children=(VBox(children=(HTML(value='<h3>u(92.0)</h3>'), Dropdown(description='Colormap:', options={…